### Импорт необходимых библиотек

In [1]:
import sqlite3

import pandas as pd
import requests

In [2]:
limit_ = 6000  # 6500 уже даёт статус 504
offset_ = 1
url = f'https://наш.дом.рф/сервисы/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'

### GET запрос для получения данных

In [3]:
res = requests.get(url)
objects_data = res.json()

In [4]:
len(objects_data['data']['list'])

6000

### Создание pandas.DataFrame

In [5]:
df = pd.json_normalize(objects_data['data']['list'])

In [6]:
df.head()

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13383,72,г Тюмень,4,20,68,2022-06-30,6289.98,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
1,13399,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,0.00,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
2,13391,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-06-30,4293.67,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
3,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
4,23682,47,д Агалатово,10,10,54,2022-06-30,2748.15,1,0,...,Общество с ограниченной ответственностью,ООО,5.562608e+09,7806536079,"Жилой комплекс ""84 высота""",NaN,NaN,NaN,NaN,NaN


### Сохранение pandas.DataFrame в excel

In [7]:
df.to_excel('df_excel.xlsx', index=False)

### Сохранение pandas.DataFrame в pickle

In [8]:
df.to_pickle('df_pickle.pkl')

### Сохранение pandas.DataFrame в базу данных

In [10]:
conn = sqlite3.connect('df_to_database')
df.to_sql('objects', conn, if_exists='replace', index=False)
conn.commit()
conn.close()